# Chapter 2: Forces（力）

この章では、ニュートンの運動法則に基づいた力のシミュレーションを学びます。

## 主要な概念

1. **ニュートンの運動法則**
   - 第1法則：慣性の法則
   - 第2法則：F = ma（力 = 質量 × 加速度）
   - 第3法則：作用・反作用の法則

2. **様々な力**
   - 重力
   - 摩擦
   - 抗力（流体抵抗）
   - 万有引力

## 例題 1: 重力の適用

物体に重力を適用し、地面でバウンドさせます。

In [ ]:
let mover;

function setup() {
  createCanvas(400, 400);
  mover = new Mover(200, 50, 2);
}

function draw() {
  background(255);
  
  // 重力（質量に比例させる）
  let gravity = createVector(0, 0.2 * mover.mass);
  mover.applyForce(gravity);
  
  mover.update();
  mover.checkEdges();
  mover.show();
}

class Mover {
  constructor(x, y, m) {
    this.position = createVector(x, y);
    this.velocity = createVector(0, 0);
    this.acceleration = createVector(0, 0);
    this.mass = m;
  }
  
  applyForce(force) {
    // F = ma → a = F/m
    let f = p5.Vector.div(force, this.mass);
    this.acceleration.add(f);
  }
  
  update() {
    this.velocity.add(this.acceleration);
    this.position.add(this.velocity);
    this.acceleration.mult(0);  // 加速度をリセット
  }
  
  checkEdges() {
    if (this.position.y > height - this.mass * 8) {
      this.position.y = height - this.mass * 8;
      this.velocity.y *= -0.9;  // エネルギー損失
    }
  }
  
  show() {
    stroke(0);
    fill(127);
    ellipse(this.position.x, this.position.y, this.mass * 16, this.mass * 16);
  }
}

## 例題 2: 複数の物体と重力

質量の異なる複数の物体が同時に落下します。

In [ ]:
let movers = [];

function setup() {
  createCanvas(400, 400);
  for (let i = 0; i < 10; i++) {
    movers.push(new Mover(random(width), 0, random(0.5, 3)));
  }
}

function draw() {
  background(255);
  
  for (let mover of movers) {
    // 重力（質量に比例）
    let gravity = createVector(0, 0.1 * mover.mass);
    mover.applyForce(gravity);
    
    // 風（一定の力）
    let wind = createVector(0.1, 0);
    mover.applyForce(wind);
    
    mover.update();
    mover.checkEdges();
    mover.show();
  }
}

class Mover {
  constructor(x, y, m) {
    this.position = createVector(x, y);
    this.velocity = createVector(0, 0);
    this.acceleration = createVector(0, 0);
    this.mass = m;
  }
  
  applyForce(force) {
    let f = p5.Vector.div(force, this.mass);
    this.acceleration.add(f);
  }
  
  update() {
    this.velocity.add(this.acceleration);
    this.position.add(this.velocity);
    this.acceleration.mult(0);
  }
  
  checkEdges() {
    let r = this.mass * 8;
    if (this.position.x > width - r) {
      this.position.x = width - r;
      this.velocity.x *= -1;
    } else if (this.position.x < r) {
      this.position.x = r;
      this.velocity.x *= -1;
    }
    if (this.position.y > height - r) {
      this.position.y = height - r;
      this.velocity.y *= -1;
    }
  }
  
  show() {
    stroke(0);
    fill(127, 200);
    ellipse(this.position.x, this.position.y, this.mass * 16, this.mass * 16);
  }
}

## 例題 3: 摩擦力

摩擦力は速度の反対方向に作用し、物体を減速させます。

In [ ]:
let movers = [];

function setup() {
  createCanvas(400, 400);
  for (let i = 0; i < 5; i++) {
    movers.push(new Mover(random(width), random(height), random(1, 4)));
  }
}

function draw() {
  background(255);
  
  for (let mover of movers) {
    // 重力
    let gravity = createVector(0, 0.1 * mover.mass);
    mover.applyForce(gravity);
    
    // 摩擦力（速度の反対方向）
    let friction = mover.velocity.copy();
    friction.normalize();
    friction.mult(-1);
    let c = 0.05;  // 摩擦係数
    friction.mult(c);
    mover.applyForce(friction);
    
    mover.update();
    mover.checkEdges();
    mover.show();
  }
}

class Mover {
  constructor(x, y, m) {
    this.position = createVector(x, y);
    this.velocity = createVector(random(-2, 2), 0);
    this.acceleration = createVector(0, 0);
    this.mass = m;
  }
  
  applyForce(force) {
    let f = p5.Vector.div(force, this.mass);
    this.acceleration.add(f);
  }
  
  update() {
    this.velocity.add(this.acceleration);
    this.position.add(this.velocity);
    this.acceleration.mult(0);
  }
  
  checkEdges() {
    let r = this.mass * 8;
    if (this.position.x > width - r) {
      this.position.x = width - r;
      this.velocity.x *= -1;
    } else if (this.position.x < r) {
      this.position.x = r;
      this.velocity.x *= -1;
    }
    if (this.position.y > height - r) {
      this.position.y = height - r;
      this.velocity.y *= -1;
    }
  }
  
  show() {
    stroke(0);
    fill(175);
    ellipse(this.position.x, this.position.y, this.mass * 16, this.mass * 16);
  }
}

## 例題 4: 流体抵抗（抗力）

物体が流体（水など）を通過するときの抵抗力をシミュレートします。

In [ ]:
let movers = [];
let liquid;

function setup() {
  createCanvas(400, 400);
  for (let i = 0; i < 9; i++) {
    movers.push(new Mover(40 + i * 40, 0, random(0.5, 3)));
  }
  liquid = new Liquid(0, height / 2, width, height / 2, 0.1);
}

function draw() {
  background(255);
  
  liquid.show();
  
  for (let mover of movers) {
    // 流体内にいるか確認
    if (liquid.contains(mover)) {
      let drag = liquid.calculateDrag(mover);
      mover.applyForce(drag);
    }
    
    // 重力
    let gravity = createVector(0, 0.1 * mover.mass);
    mover.applyForce(gravity);
    
    mover.update();
    mover.checkEdges();
    mover.show();
  }
}

function mousePressed() {
  for (let mover of movers) {
    mover.position.y = 0;
    mover.velocity.set(0, 0);
  }
}

class Liquid {
  constructor(x, y, w, h, c) {
    this.x = x;
    this.y = y;
    this.w = w;
    this.h = h;
    this.c = c;  // 抗力係数
  }
  
  contains(mover) {
    let pos = mover.position;
    return pos.x > this.x && pos.x < this.x + this.w &&
           pos.y > this.y && pos.y < this.y + this.h;
  }
  
  calculateDrag(mover) {
    let speed = mover.velocity.mag();
    let dragMagnitude = this.c * speed * speed;
    
    let drag = mover.velocity.copy();
    drag.mult(-1);
    drag.normalize();
    drag.mult(dragMagnitude);
    
    return drag;
  }
  
  show() {
    noStroke();
    fill(175, 200);
    rect(this.x, this.y, this.w, this.h);
  }
}

class Mover {
  constructor(x, y, m) {
    this.position = createVector(x, y);
    this.velocity = createVector(0, 0);
    this.acceleration = createVector(0, 0);
    this.mass = m;
  }
  
  applyForce(force) {
    let f = p5.Vector.div(force, this.mass);
    this.acceleration.add(f);
  }
  
  update() {
    this.velocity.add(this.acceleration);
    this.position.add(this.velocity);
    this.acceleration.mult(0);
  }
  
  checkEdges() {
    let r = this.mass * 8;
    if (this.position.y > height - r) {
      this.position.y = height - r;
      this.velocity.y *= -0.9;
    }
  }
  
  show() {
    stroke(0);
    fill(127);
    ellipse(this.position.x, this.position.y, this.mass * 16, this.mass * 16);
  }
}

## 例題 5: 万有引力

ニュートンの万有引力の法則：F = G × m1 × m2 / r²

In [ ]:
let movers = [];
let attractor;

function setup() {
  createCanvas(400, 400);
  for (let i = 0; i < 10; i++) {
    movers.push(new Mover(random(width), random(height), random(0.5, 2)));
  }
  attractor = new Attractor();
}

function draw() {
  background(255);
  
  attractor.show();
  
  for (let mover of movers) {
    let force = attractor.attract(mover);
    mover.applyForce(force);
    
    mover.update();
    mover.show();
  }
}

class Attractor {
  constructor() {
    this.position = createVector(width / 2, height / 2);
    this.mass = 20;
    this.G = 1;  // 万有引力定数
  }
  
  attract(mover) {
    let force = p5.Vector.sub(this.position, mover.position);
    let distance = force.mag();
    // 距離を制限（極端な値を避ける）
    distance = constrain(distance, 5, 25);
    
    force.normalize();
    let strength = (this.G * this.mass * mover.mass) / (distance * distance);
    force.mult(strength);
    
    return force;
  }
  
  show() {
    stroke(0);
    fill(175, 200);
    ellipse(this.position.x, this.position.y, this.mass * 2, this.mass * 2);
  }
}

class Mover {
  constructor(x, y, m) {
    this.position = createVector(x, y);
    this.velocity = createVector(random(-1, 1), random(-1, 1));
    this.acceleration = createVector(0, 0);
    this.mass = m;
  }
  
  applyForce(force) {
    let f = p5.Vector.div(force, this.mass);
    this.acceleration.add(f);
  }
  
  update() {
    this.velocity.add(this.acceleration);
    this.position.add(this.velocity);
    this.acceleration.mult(0);
  }
  
  show() {
    stroke(0);
    fill(127);
    ellipse(this.position.x, this.position.y, this.mass * 16, this.mass * 16);
  }
}

## 例題 6: N体問題

複数の物体が互いに引力を及ぼし合うシミュレーションです。

In [ ]:
let bodies = [];

function setup() {
  createCanvas(400, 400);
  for (let i = 0; i < 10; i++) {
    bodies.push(new Body(random(width), random(height), random(5, 20)));
  }
}

function draw() {
  background(255);
  
  // すべての物体ペアで引力を計算
  for (let i = 0; i < bodies.length; i++) {
    for (let j = 0; j < bodies.length; j++) {
      if (i !== j) {
        let force = bodies[j].attract(bodies[i]);
        bodies[i].applyForce(force);
      }
    }
  }
  
  for (let body of bodies) {
    body.update();
    body.show();
  }
}

class Body {
  constructor(x, y, m) {
    this.position = createVector(x, y);
    this.velocity = createVector(random(-1, 1), random(-1, 1));
    this.acceleration = createVector(0, 0);
    this.mass = m;
    this.G = 0.4;
  }
  
  attract(body) {
    let force = p5.Vector.sub(this.position, body.position);
    let distance = constrain(force.mag(), 5, 25);
    
    force.normalize();
    let strength = (this.G * this.mass * body.mass) / (distance * distance);
    force.mult(strength);
    
    return force;
  }
  
  applyForce(force) {
    let f = p5.Vector.div(force, this.mass);
    this.acceleration.add(f);
  }
  
  update() {
    this.velocity.add(this.acceleration);
    this.position.add(this.velocity);
    this.acceleration.mult(0);
  }
  
  show() {
    stroke(0);
    fill(127, 150);
    ellipse(this.position.x, this.position.y, this.mass, this.mass);
  }
}

## まとめ

### ニュートンの運動法則
- **第1法則**：物体は外力を受けない限り、静止または等速直線運動を続ける
- **第2法則**：F = ma（加速度は力に比例し、質量に反比例）
- **第3法則**：作用・反作用は等しい大きさで逆向き

### 力の公式
- **摩擦力**：Ff = -μ × N × v̂（速度の反対方向）
- **抗力**：Fd = -½ × ρ × v² × A × Cd × v̂（速度の2乗に比例）
- **万有引力**：Fg = G × m1 × m2 / r²（距離の2乗に反比例）

### 実装のポイント
- 毎フレーム加速度をリセットする
- 力を適用する際は質量で割る
- 距離は適切な範囲に制限する